
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/ipl_2021_matches.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:


temp_table_name = "ipl_2021_matches_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:

permanent_table_name = "ipl_2021_matches_csv"

In [0]:
ipl_path = "/FileStore/tables/ipl_2021_matches.csv"
string_schema = """
    year INT,
    series_type STRING,
    series_name STRING,
    match_no STRING,
    match_type STRING,
    match_name STRING,
    match_href STRING,
    match_team1 STRING,
    match_team2 STRING,
    match_datetime_start STRING,
    match_date_end STRING,
    match_venue STRING
"""

ipl_df_ddl = spark.read.option("header", "true").schema(string_schema).csv(ipl_path)
ipl_df_ddl.show(5)
ipl_df_ddl.printSchema()


+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|        T20|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|https://www.cricb...|       null|       null|     

In [0]:
from pyspark.sql.types import *

struct_schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("series_type", StringType(), True),
    StructField("series_name", StringType(), True),
    StructField("match_no", StringType(), True),
    StructField("match_type", StringType(), True),
    StructField("match_name", StringType(), True),
    StructField("match_href", StringType(), True),
    StructField("match_team1", StringType(), True),
    StructField("match_team2", StringType(), True),
    StructField("match_datetime_start", StringType(), True),
    StructField("match_date_end", StringType(), True),
    StructField("match_venue", StringType(), True)
])

ipl_df_struct = spark.read.option("header", "true").schema(struct_schema).csv(ipl_path)
ipl_df_struct.show(5)
ipl_df_struct.printSchema()


+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|        T20|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|https://www.cricb...|       null|       null|     

In [0]:
ipl_cleaned_df = ipl_df_struct.dropna(subset=["match_no", "match_team1", "match_team2"])
ipl_cleaned_df.show(5)


+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|series_name|match_no|match_type|match_name|match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+



In [0]:
ipl_cleaned_df.write.mode("overwrite").parquet("/FileStore/ipl_cleaned.parquet")


In [0]:
ipl_parquet_df = spark.read.parquet("/FileStore/ipl_cleaned.parquet")
ipl_parquet_df.show(5)
ipl_parquet_df.printSchema()


+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|series_name|match_no|match_type|match_name|match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+

root
 |-- year: integer (nullable = true)
 |-- series_type: string (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_name: string (nullable = true)
 |-- match_href: string (nullable = true)
 |-- match_team1: string (nullable = true)
 |-- match_team2: string (nullable = true)
 |-- match_datetime_start: string (null

In [0]:
ipl_parquet_df.groupBy("match_team1").count().orderBy("count", ascending=False).show()

ipl_parquet_df.select("match_venue").distinct().show()

ipl_parquet_df.select("year").distinct().show()


+-----------+-----+
|match_team1|count|
+-----------+-----+
+-----------+-----+

+-----------+
|match_venue|
+-----------+
+-----------+

+----+
|year|
+----+
+----+



In [0]:
ipl_cleaned_df.show(5)
ipl_cleaned_df.printSchema()


+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|series_name|match_no|match_type|match_name|match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+

root
 |-- year: integer (nullable = true)
 |-- series_type: string (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_name: string (nullable = true)
 |-- match_href: string (nullable = true)
 |-- match_team1: string (nullable = true)
 |-- match_team2: string (nullable = true)
 |-- match_datetime_start: string (null

In [0]:
ipl_df = spark.read.parquet("/FileStore/ipl_cleaned.parquet")
ipl_df.show(5)


+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|series_name|match_no|match_type|match_name|match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+
+----+-----------+-----------+--------+----------+----------+----------+-----------+-----------+--------------------+--------------+-----------+



In [0]:
display(dbutils.fs.ls("/FileStore/"))



path,name,size,modificationTime
dbfs:/FileStore/ipl_cleaned.parquet/,ipl_cleaned.parquet/,0,0
dbfs:/FileStore/tables/,tables/,0,0


In [0]:

ipl_df = spark.table("default.ipl_2021_matches_1_csv")
ipl_df.show(5)
ipl_df.printSchema()


+----+-------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|ht

In [0]:

ipl_df = spark.table("default.ipl_2021_matches_1_csv")
ipl_df.show(5)
ipl_df.printSchema()


+----+-------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|ht

In [0]:
# Load the data from the created table
ipl_df = spark.table("default.ipl_2021_matches_1_csv")
ipl_df.show(5)
ipl_df.printSchema()


+----+-------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|ht

In [0]:
ipl_df = spark.read.option("header", True).option("inferSchema", False).csv("/FileStore/tables/ipl_2021_matches-2.csv")
ipl_df.printSchema()
ipl_df.show(5, truncate=False)


root
 |-- year: string (nullable = true)
 |-- series_type: string (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_name: string (nullable = true)
 |-- match_href: string (nullable = true)
 |-- match_team1: string (nullable = true)
 |-- match_team2: string (nullable = true)
 |-- match_datetime_start: string (nullable = true)
 |-- match_date_end: string (nullable = true)
 |-- match_venue: string (nullable = true)

+----+-----------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|series_name               |match_no|match_type|match_name                                              |match_href                                                          

In [0]:
ipl_df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/ipl_2021_matches-2.csv")
ipl_df.printSchema()
ipl_df.show(5, truncate=False)


root
 |-- year: integer (nullable = true)
 |-- series_type: timestamp (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_name: string (nullable = true)
 |-- match_href: string (nullable = true)
 |-- match_team1: string (nullable = true)
 |-- match_team2: string (nullable = true)
 |-- match_datetime_start: string (nullable = true)
 |-- match_date_end: string (nullable = true)
 |-- match_venue: string (nullable = true)

+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type        |series_name               |match_no|match_type|match_name                                              |match_href                                      

In [0]:
from pyspark.sql.functions import split, trim

ipl_df = ipl_df.withColumn("match_team1", trim(split(ipl_df.match_name, " vs ")[0]))
ipl_df = ipl_df.withColumn("match_team2", trim(split(split(ipl_df.match_name, " vs ")[1], ",")[0]))
ipl_df.select("match_name", "match_team1", "match_team2").show(5, truncate=False)


+--------------------------------------------------------+---------------------+---------------------------+
|match_name                                              |match_team1          |match_team2                |
+--------------------------------------------------------+---------------------+---------------------------+
|MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU, 1st Match|MUMBAI INDIANS       |ROYAL CHALLENGERS BENGALURU|
|CHENNAI SUPER KINGS vs DELHI CAPITALS, 2nd Match        |CHENNAI SUPER KINGS  |DELHI CAPITALS             |
|SUNRISERS HYDERABAD vs KOLKATA KNIGHT RIDERS, 3rd Match |SUNRISERS HYDERABAD  |KOLKATA KNIGHT RIDERS      |
|RAJASTHAN ROYALS vs PUNJAB KINGS, 4th Match             |RAJASTHAN ROYALS     |PUNJAB KINGS               |
|KOLKATA KNIGHT RIDERS vs MUMBAI INDIANS, 5th Match      |KOLKATA KNIGHT RIDERS|MUMBAI INDIANS             |
+--------------------------------------------------------+---------------------+---------------------------+
only showing top 5 

In [0]:
from pyspark.sql.functions import to_timestamp

ipl_df = ipl_df.withColumn("match_datetime_start", to_timestamp(ipl_df["series_type"], "yyyy-MM-dd HH:mm:ss"))


In [0]:
ipl_df.write.mode("overwrite").parquet("/FileStore/ipl_cleaned.parquet")


In [0]:
ipl_df.select("match_team1", "match_team2").distinct().show(truncate=False)


+---------------------------+---------------------------+
|match_team1                |match_team2                |
+---------------------------+---------------------------+
|SUNRISERS HYDERABAD        |KOLKATA KNIGHT RIDERS      |
|PUNJAB KINGS               |KOLKATA KNIGHT RIDERS      |
|RAJASTHAN ROYALS           |DELHI CAPITALS             |
|DELHI CAPITALS             |MUMBAI INDIANS             |
|RAJASTHAN ROYALS           |KOLKATA KNIGHT RIDERS      |
|MUMBAI INDIANS             |SUNRISERS HYDERABAD        |
|CHENNAI SUPER KINGS        |DELHI CAPITALS             |
|CHENNAI SUPER KINGS        |RAJASTHAN ROYALS           |
|PUNJAB KINGS               |MUMBAI INDIANS             |
|KOLKATA KNIGHT RIDERS      |MUMBAI INDIANS             |
|ROYAL CHALLENGERS BENGALURU|RAJASTHAN ROYALS           |
|RAJASTHAN ROYALS           |PUNJAB KINGS               |
|PUNJAB KINGS               |SUNRISERS HYDERABAD        |
|ROYAL CHALLENGERS BENGALURU|KOLKATA KNIGHT RIDERS      |
|DELHI CAPITAL

In [0]:
from pyspark.sql.functions import col

ipl_df.select("match_team1").groupBy("match_team1").count().orderBy("count", ascending=False).show()
ipl_df.select("match_team2").groupBy("match_team2").count().orderBy("count", ascending=False).show()


+--------------------+-----+
|         match_team1|count|
+--------------------+-----+
|      DELHI CAPITALS|    9|
| CHENNAI SUPER KINGS|    8|
|ROYAL CHALLENGERS...|    8|
| SUNRISERS HYDERABAD|    8|
|KOLKATA KNIGHT RI...|    8|
|    RAJASTHAN ROYALS|    7|
|      MUMBAI INDIANS|    7|
|        PUNJAB KINGS|    7|
+--------------------+-----+

+--------------------+-----+
|         match_team2|count|
+--------------------+-----+
|KOLKATA KNIGHT RI...|   10|
| CHENNAI SUPER KINGS|    8|
|ROYAL CHALLENGERS...|    8|
|      MUMBAI INDIANS|    8|
|    RAJASTHAN ROYALS|    7|
| SUNRISERS HYDERABAD|    7|
|      DELHI CAPITALS|    7|
|        PUNJAB KINGS|    7|
+--------------------+-----+



In [0]:
from pyspark.sql.functions import expr

team_count_df = (
    ipl_df.select(col("match_team1").alias("team"))
    .union(ipl_df.select(col("match_team2").alias("team")))
    .groupBy("team").count().orderBy("count", ascending=False)
)

team_count_df.show()


+--------------------+-----+
|                team|count|
+--------------------+-----+
|KOLKATA KNIGHT RI...|   18|
| CHENNAI SUPER KINGS|   16|
|ROYAL CHALLENGERS...|   16|
|      DELHI CAPITALS|   16|
| SUNRISERS HYDERABAD|   15|
|      MUMBAI INDIANS|   15|
|    RAJASTHAN ROYALS|   14|
|        PUNJAB KINGS|   14|
+--------------------+-----+



In [0]:
ipl_df.write.mode("overwrite").parquet("/FileStore/ipl_final_cleaned.parquet")


In [0]:
ipl_cleaned_df = spark.read.parquet("/FileStore/ipl_cleaned.parquet")
ipl_cleaned_df.show(5)


+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|         match_team1|         match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|      MUMBAI INDIANS|ROYAL CHALLENGERS...| 2025-04-09 20:00:00|          null|       null|
|2021|2025-04-09 20:00:00|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...| CHENNAI SUPER KINGS|      DELHI CAPITALS| 2025-04-09 20:00:00|          null|       null|


In [0]:
ipl_cleaned_df.createOrReplaceTempView("ipl_matches")


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

ipl_schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("series_type", StringType(), True),
    StructField("series_name", StringType(), True),
    StructField("match_no", StringType(), True),
    StructField("match_type", StringType(), True),
    StructField("match_name", StringType(), True),
    StructField("match_href", StringType(), True),
    StructField("match_team1", StringType(), True),
    StructField("match_team2", StringType(), True),
    StructField("match_datetime_start", StringType(), True),
    StructField("match_date_end", StringType(), True),
    StructField("match_venue", StringType(), True)
])


In [0]:
ipl_df_struct = spark.read.schema(ipl_schema).option("header", True).csv("/FileStore/tables/ipl_2021_matches-2.csv")
ipl_df_struct.show(5)
ipl_df_struct.printSchema()


+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|         series_name|match_no|match_type|          match_name|          match_href|match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+--------------------+--------+----------+--------------------+--------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|        T20|Indian Premier Le...|    null|    League|MUMBAI INDIANS vs...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|CHENNAI SUPER KIN...|https://www.cricb...|       null|       null|                null|          null|       null|
|2021|        T20|Indian Premier Le...|    null|    League|SUNRISERS HYDERAB...|https://www.cricb...|       null|       null|     

In [0]:
ipl_validated_df = ipl_df_struct \
    .filter(col("year") == 2021) \
    .filter(col("match_team1").isNotNull()) \
    .filter(col("match_team2").isNotNull()) \
    .filter(col("match_venue").isNotNull()) \
    .filter(col("match_type").isin("League", "Playoff", "Final"))


In [0]:
ipl_validated_df.write.mode("overwrite").parquet("/FileStore/ipl_validated.parquet")
ipl_validated_df.createOrReplaceTempView("ipl_matches_validated")


In [0]:
ipl_cleaned_df.createOrReplaceTempView("ipl_matches")
spark.sql("SELECT DISTINCT year FROM ipl_matches").show()

spark.sql("""
    SELECT match_team1, COUNT(*) as count 
    FROM ipl_matches 
    GROUP BY match_team1 
    ORDER BY count DESC
""").show()


+----+
|year|
+----+
|2021|
+----+

+--------------------+-----+
|         match_team1|count|
+--------------------+-----+
|      DELHI CAPITALS|    9|
| CHENNAI SUPER KINGS|    8|
|ROYAL CHALLENGERS...|    8|
| SUNRISERS HYDERABAD|    8|
|KOLKATA KNIGHT RI...|    8|
|    RAJASTHAN ROYALS|    7|
|      MUMBAI INDIANS|    7|
|        PUNJAB KINGS|    7|
+--------------------+-----+



In [0]:
import pandas as pd
import sqlite3

ipl_pd_df = ipl_cleaned_df.toPandas()

conn = sqlite3.connect("ipl_data.db")
ipl_pd_df.to_sql("match_data", conn, if_exists="replace", index=False)

query = "SELECT match_team1, COUNT(*) FROM match_data GROUP BY match_team1"
result = pd.read_sql_query(query, conn)
print(result)

conn.close()


                   match_team1  COUNT(*)
0          CHENNAI SUPER KINGS         8
1               DELHI CAPITALS         9
2        KOLKATA KNIGHT RIDERS         8
3               MUMBAI INDIANS         7
4                 PUNJAB KINGS         7
5             RAJASTHAN ROYALS         7
6  ROYAL CHALLENGERS BENGALURU         8
7          SUNRISERS HYDERABAD         8
